In [1]:
import basedosdados as bd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
import seaborn as sns

In [2]:
query1 = 'SELECT * FROM `basedosdados.br_ms_sim.dicionario`'

In [3]:
query2 = 'SELECT * FROM `basedosdados.br_ms_sim.microdados` WHERE sigla_uf = "RJ" AND ano = 2019'

In [4]:
df_dicionario = bd.read_sql(query1, billing_project_id='sim-project-360116')

Downloading: 100%|███████████████████████████████████████████████████████████████| 572/572 [00:00<00:00, 1109.07rows/s]


In [5]:
df_micro = bd.read_sql(query2, billing_project_id = 'sim-project-360116')

Downloading: 100%|█████████████████████████████████████████████████████████| 144600/144600 [02:05<00:00, 1148.49rows/s]


In [6]:
# Pegando apenas os dados de homicídios
df_homicide = df_micro[df_micro['circunstancia_obito']=='3']

In [7]:
# Criando uma coluna pra contabilizar as mortes
df_homicide['numero_homicidio'] = 1

In [8]:
# Agrupando por município e somando o número de homicídios
teste = df_homicide.groupby(['id_municipio_ocorrencia']).agg({"numero_homicidio":"sum"})
teste

,numero_homicidio
id_municipio_ocorrencia,
2103752,1
2913606,1
2923001,1
3127701,1
3147808,1
...,...
4106902,2
5005202,1
5205497,1


In [9]:
teste = teste.sort_values(by='numero_homicidio', ascending=False)
teste

,numero_homicidio
id_municipio_ocorrencia,
3304557,619
3301702,436
3304904,251
3301009,149
3300100,137
...,...
3303104,1
2913606,1
3304102,1


In [10]:
idade = df_homicide[["idade","raca_cor"]]
idade['raca_cor'] = idade['raca_cor'].fillna("9")

In [41]:
branca_age = idade[idade["raca_cor"]=="1"]
preta_age = idade[idade["raca_cor"]=="2"]
amarela_age = idade[idade["raca_cor"]=="3"]
parda_age = idade[idade["raca_cor"]=="4"]
indigena_age = idade[idade["raca_cor"]=="5"]
raca_n_age = idade[idade["raca_cor"]=="9"]

In [12]:
# Nessas colunas temos alguns missing values (None). Para lidar com isso, preenchemos no lugar o número para facilitar a contabilização
df_homicide['raca_cor'] = df_homicide['raca_cor'].fillna(9)
df_homicide['sexo'] = df_homicide['sexo'].fillna(9)

In [13]:
# Criando colunas por sexo
df_homicide['Homem'] = np.where(df_homicide['sexo']=="1",1,0)
df_homicide['Mulher'] = np.where(df_homicide['sexo']=="2",1,0)
df_homicide['genero_nao_decl'] = np.where(df_homicide['sexo']==9,1,0)

In [14]:
#Criando colunas por raça
df_homicide['branca'] = np.where(df_homicide['raca_cor']=="1",1,0) 
df_homicide['preta'] = np.where(df_homicide['raca_cor']=="2",1,0) 
df_homicide['amarela'] = np.where(df_homicide['raca_cor']=="3",1,0) 
df_homicide['parda'] = np.where(df_homicide['raca_cor']=="4",1,0) 
df_homicide['indigena'] = np.where(df_homicide['raca_cor']=="5",1,0) 
df_homicide['raca_nao_decl'] = np.where(df_homicide['raca_cor']==9,1,0)

In [15]:
df_homicide

,ano,sigla_uf,sequencial_obito,tipo_obito,causa_basica,data_obito,hora_obito,naturalidade,data_nascimento,idade,...,numero_homicidio,Homem,Mulher,genero_nao_decl,branca,preta,amarela,parda,indigena,raca_nao_decl
8,2019,RJ,1254134,2,X959,2019-08-12,06:19:00,800,NaT,NaN,...,1,1,0,0,0,0,0,1,0,0
52,2019,RJ,1194015,2,X959,2019-08-13,11:00:00,None,1985-09-28,33.0,...,1,1,0,0,0,0,0,1,0,0
104,2019,RJ,1324191,2,X954,2019-10-20,None,None,1974-07-10,45.0,...,1,1,0,0,0,0,0,1,0,0
198,2019,RJ,1225246,2,X954,2019-10-20,None,None,1986-01-20,33.0,...,1,1,0,0,0,1,0,0,0,0
199,2019,RJ,1292375,2,Y094,2019-07-29,None,None,1952-02-26,67.0,...,1,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144444,2019,RJ,808002,2,X958,2019-03-30,None,852,1987-03-25,32.0,...,1,1,0,0,0,0,0,0,0,1
144453,2019,RJ,313690,2,X959,2019-08-06,20:20:00,852,1982-06-12,37.0,...,1,1,0,0,1,0,0,0,0,0
144511,2019,RJ,222371,2,X995,2019-11-17,18:00:00,853,1975-12-22,43.0,...,1,1,0,0,0,0,0,1,0,0
144514,2019,RJ,668407,2,X959,2019-03-11,21:39:00,853,1988-03-08,31.0,...,1,1,0,0,1,0,0,0,0,0


In [16]:
# Agrupando por município e somando o número de homicídios por gênero
Homem = df_homicide.groupby(['id_municipio_ocorrencia']).agg({"Homem":"sum"})
Mulher = df_homicide.groupby(['id_municipio_ocorrencia']).agg({"Mulher":"sum"})
gen_n = df_homicide.groupby(['id_municipio_ocorrencia']).agg({"genero_nao_decl":"sum"})

In [17]:
# Agrupando por município e somando o número de homicídios por raça
branca = df_homicide.groupby(['id_municipio_ocorrencia']).agg({"branca":"sum"})
preta = df_homicide.groupby(['id_municipio_ocorrencia']).agg({"preta":"sum"})
amarela = df_homicide.groupby(['id_municipio_ocorrencia']).agg({"amarela":"sum"})
parda = df_homicide.groupby(['id_municipio_ocorrencia']).agg({"parda":"sum"})
indigena = df_homicide.groupby(['id_municipio_ocorrencia']).agg({"indigena":"sum"})
raca_n = df_homicide.groupby(['id_municipio_ocorrencia']).agg({"raca_nao_decl":"sum"})

In [18]:
# Organizando de forma decrescente
Homem = Homem.sort_values(by="Homem",ascending=False)
Mulher = Mulher.sort_values(by="Mulher",ascending=False)
gen_n = gen_n.sort_values(by="genero_nao_decl",ascending=False)

In [19]:
# Organizando de forma decrescente
branca = branca.sort_values(by="branca",ascending=False)
preta = preta.sort_values(by="preta",ascending=False)
amarela = amarela.sort_values(by="amarela",ascending=False)
parda = parda.sort_values(by="parda",ascending=False)
indigena = indigena.sort_values(by="indigena",ascending=False)
raca_n = raca_n.sort_values(by="raca_nao_decl",ascending=False)

In [20]:
#Criando nova coluna para realizar o merge
Homem['muni_ocorrencia'] = Homem.index
Mulher['muni_ocorrencia'] = Mulher.index
gen_n['muni_ocorrencia'] = gen_n.index

In [21]:
# Criando nova coluna para realziar o merge
branca['muni_ocorrencia'] = branca.index
preta['muni_ocorrencia'] = preta.index
amarela['muni_ocorrencia'] = amarela.index
parda['muni_ocorrencia'] = parda.index
indigena['muni_ocorrencia'] = indigena.index
raca_n['muni_ocorrencia'] = raca_n.index

In [22]:
# Realizando o merge
xx = pd.merge(Homem,Mulher,how='inner',on='muni_ocorrencia')
genero_df = pd.merge(xx,gen_n,how='inner',on='muni_ocorrencia')

In [23]:
xy = pd.merge(branca,preta,how='inner',on='muni_ocorrencia')
yx = pd.merge(amarela,parda,how='inner',on='muni_ocorrencia')
xyz = pd.merge(indigena,raca_n,how='inner',on='muni_ocorrencia')
xw = pd.merge(xy,yx,how='inner',on='muni_ocorrencia')
raca_df = pd.merge(xw,xyz,how='inner',on='muni_ocorrencia')

In [24]:
teste['muni_ocorrencia'] = teste.index


,numero_homicidio,muni_ocorrencia
id_municipio_ocorrencia,,
3304557,619,3304557
3301702,436,3301702
3304904,251,3304904
3301009,149,3301009
3300100,137,3300100
...,...,...
3303104,1,3303104
2913606,1,2913606
3304102,1,3304102


In [25]:
df_incomplete = pd.merge(teste,raca_df,how='inner',on='muni_ocorrencia')
df_incomplete

,numero_homicidio,muni_ocorrencia,branca,preta,amarela,parda,indigena,raca_nao_decl
0,619,3304557,169,90,2,343,1,14
1,436,3301702,83,66,0,286,0,1
2,251,3304904,62,36,0,152,0,1
3,149,3301009,17,40,0,90,1,1
4,137,3300100,53,13,0,70,0,1
...,...,...,...,...,...,...,...,...
94,1,3303104,1,0,0,0,0,0
95,1,2913606,0,0,0,1,0,0
96,1,3304102,1,0,0,0,0,0
97,1,3304508,1,0,0,0,0,0


In [61]:
data = pd.merge(df_incomplete,genero_df,how='inner',on='muni_ocorrencia')


,numero_homicidio,muni_ocorrencia,branca,preta,amarela,parda,indigena,raca_nao_decl,Homem,Mulher,genero_nao_decl
0,619,3304557,169,90,2,343,1,14,555,62,0
1,436,3301702,83,66,0,286,0,1,411,23,0
2,251,3304904,62,36,0,152,0,1,238,13,0
3,149,3301009,17,40,0,90,1,1,141,8,0
4,137,3300100,53,13,0,70,0,1,133,4,0
...,...,...,...,...,...,...,...,...,...,...,...
94,1,3303104,1,0,0,0,0,0,1,0,0
95,1,2913606,0,0,0,1,0,0,1,0,0
96,1,3304102,1,0,0,0,0,0,0,1,0
97,1,3304508,1,0,0,0,0,0,1,0,0


In [83]:
data[:20]

,numero_homicidio,muni_ocorrencia,branca,preta,amarela,parda,indigena,raca_nao_decl,Homem,Mulher
0,619,3304557,169,90,2,343,1,14,555,62
1,436,3301702,83,66,0,286,0,1,411,23
2,251,3304904,62,36,0,152,0,1,238,13
3,149,3301009,17,40,0,90,1,1,141,8
4,137,3300100,53,13,0,70,0,1,133,4
5,126,3305109,27,21,0,78,0,0,123,3
6,113,3300704,37,28,1,47,0,0,111,2
7,105,3301900,28,17,0,51,0,9,96,3
8,103,3302403,20,27,0,56,0,0,96,7
9,102,3303302,29,24,1,47,0,1,93,8


In [ ]:
df2 = pd.DataFrame(np.array([["Rio de Janeiro", 3304557], ["Sao Goncalo", 3304904],
                             ["Campos dos Goytacazes",3301009],["Angra dos Reis",3300100], ["Sao Joao de Meriti",3305109], 
                             ["Araruama",3300209], ["Resende",3304201],["Belford Roxo",3300456], ["Rio das Ostras",3304524],
                             ["Barra Mansa",3300407], ["Tres Rios",3306008],["Nova Iguacu",3303500], ["Sao Pedro da Aldeia", 3305208], 
                             ["Mage", 3302502], ["Volta Redonda",3306305],["Niteroi", 3303302], ["Macae",3302403], ["Itaborai",3301900], 
                             ["Cabo Frio",3300704],["Duque de Caxias",3301702]]),
                   columns=['municipio', 'muni_ocorrencia'])

In [ ]:
data2 = pd.merge(data,df2,how='inner',on='muni_ocorrencia')

In [ ]:
teste2 = pd.merge(teste,df2,how='inner',on='muni_ocorrencia')

In [112]:
# Plotando os gráficos
teste2.plot(kind='bar',
           figsize=(13,5),
           stacked =True,
           linewidth = 1)


plt.xticks(list(range(0,20)),teste2["municipio"])


plt.legend(loc="best",prop = {"size" : 13})
plt.title("Homicídios por Municípios do Rio de Janeiro")
plt.ylabel("Homicídios")
plt.xlabel("Municípios")
#plt.show()
plt.savefig('homicidio_plot.jpg')
plt.close()

In [110]:
# Gráfico por raça sem considerar os casos não identificados
data2[["branca","preta","amarela","parda","indigena"]].plot(kind='bar',
           figsize=(13,5),
           stacked =True,
           linewidth = 1)

plt.xticks(list(range(0,20)),data2["municipio"])

plt.legend(loc="best",prop = {"size" : 13})
plt.title("Homicídios por Municípios do Rio de Janeiro")
plt.ylabel("Homicídios")
plt.xlabel("Municípios")
#plt.show()
plt.savefig('genero1_barplot.jpg')
plt.close()

In [109]:
# Gráfico por raça contando os casos não identificados
data2[["branca","preta","amarela","parda","indigena","raca_nao_decl"]].plot(kind='bar',
           figsize=(13,5),
           stacked =True,
           linewidth = 1)

plt.xticks(list(range(0,20)),data2["municipio"])

plt.legend(loc="best",prop = {"size" : 13})
plt.title("Homicídios por Municípios do Rio de Janeiro")
plt.ylabel("Homicídios")
plt.xlabel("Municípios")
#plt.show()
plt.savefig('raca_barplot.jpg')
plt.close()

In [108]:
data[["Homem","Mulher"]][:20].plot(kind='bar', figsize=(13,5),
         stacked = True, linewidth=1)

plt.legend(loc="best",prop = {"size" : 13})
plt.title("Proporção das vítimas de homicídio por gênero (2019)")
plt.ylabel("Homicídios")
plt.xlabel("Municípios")
#plt.show()
plt.savefig('genero_barplot.jpg')
plt.close()

In [107]:
plt.figure(figsize=(8,8))
df_homicide["sexo"].value_counts().plot.pie(autopct = "%1.0f%%", # Mostrar o valor percentual
                                             colors =sns.color_palette("rainbow",3),
                                             wedgeprops = {"linewidth":2,"edgecolor":"white"})

my_circ = plt.Circle((0,0),.7,color = "white") # Experimenta tirar essa linha do plot
plt.gca().add_artist(my_circ)


plt.title("Proporção por Gênero")
#plt.show()
plt.savefig('pie_plotage.jpg')
plt.close()

In [75]:
data = data.drop(['genero_nao_decl'],axis=1)

In [77]:
data[:20].corr()

,numero_homicidio,branca,preta,amarela,parda,indigena,raca_nao_decl,Homem,Mulher
numero_homicidio,1.000000,0.963239,0.962089,0.626970,0.992917,0.580570,0.670044,0.999264,0.925603
branca,0.963239,1.000000,0.892005,0.724789,0.930634,0.542679,0.733465,0.957855,0.938747
preta,0.962089,0.892005,1.000000,0.649787,0.952260,0.661561,0.610972,0.961445,0.891928
amarela,0.626970,0.724789,0.649787,1.000000,0.547658,0.522976,0.644045,0.611121,0.735477
parda,0.992917,0.930634,0.952260,0.547658,1.000000,0.547710,0.613041,0.994981,0.892407
indigena,0.580570,0.542679,0.661561,0.522976,0.547710,1.000000,0.571551,0.568527,0.662612
raca_nao_decl,0.670044,0.733465,0.610972,0.644045,0.613041,0.571551,1.000000,0.651159,0.752871
Homem,0.999264,0.957855,0.961445,0.611121,0.994981,0.568527,0.651159,1.000000,0.910890
Mulher,0.925603,0.938747,0.891928,0.735477,0.892407,0.662612,0.752871,0.910890,1.000000


In [106]:
correlation = data[:20].corr()
plt.figure(figsize=(13,8))
sns.heatmap(correlation,annot=True,fmt="f",linecolor="k",
            linewidths=2,cmap =sns.color_palette("Set2"))
plt.title("Correlação Entre as Variáveis")
#plt.show()
plt.savefig('correlation1.jpg')
plt.close()

In [105]:
# Plot de idade
plt.figure(figsize=(13,5))
sns.violinplot(df_homicide["raca_cor"],df_homicide["idade"],palette="rainbow")
plt.title("Idade por Raça")
plt.xticks(rotation = 50)
#plt.show()
plt.savefig('violin_age1.jpg')
plt.close()

In [104]:
# Plots dos histogramas das idades
sns.distplot(branca_age[['idade']], hist=False, rug=True)
sns.distplot(amarela_age[['idade']], hist=False, rug=True)
sns.distplot(parda_age[['idade']], hist=False, rug=True)
sns.distplot(raca_n_age[['idade']], hist=True, rug=True)
plt.title("Densidade da idade por raça")
#plt.show()

plt.savefig('dens_age.jpg')
plt.close()

In [87]:
data2.to_csv("df_data2BD.csv")

In [101]:
teste2.to_csv("df_teste2BD.csv")

In [89]:
data.to_csv("df_dataBD.csv")

In [90]:
teste.to_csv("df_testeBD.csv")

In [113]:
df_homicide.to_csv("homicide.csv")